<a href="https://colab.research.google.com/github/O-Kpy/Kaggle/blob/main/Tabular_Playground_Jun2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from scipy.stats import skew

In [ ]:
submit = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/sample_submission.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/test.csv')
train = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/train.csv')
train.shape, test.shape, submit.shape

((200000, 77), (100000, 76), (100000, 10))

In [ ]:
train.target.value_counts().sort_index(ascending=False).plot.bar()

In [ ]:
data=pd.concat([train, test], axis=0)

In [ ]:
data.isnull().sum().sort_values(ascending=False)
# Null값 없고

In [ ]:
data.skew().sort_values(ascending=False)
# 스케일링 해야하고

feature_74    5.353961
feature_47    5.217023
feature_15    4.612932
feature_24    4.391702
feature_32    4.214856
                ...   
feature_43    0.905721
feature_54    0.905510
feature_19    0.897660
feature_12    0.749524
id            0.000000
Length: 76, dtype: float64

In [ ]:
for i in data.drop(columns=['id', 'target']).columns:
    if data[i].skew() > 0.5:
        data[i] = np.log1p(data[i]+1)

In [ ]:
data_agg = data.drop(columns='target').groupby('id').agg(['min', 'max', 'count', 'mean'])
data_agg_col = []

for i in data_agg.columns.levels[0]:
    for j in data_agg.columns.levels[1]:
        data_agg_col.append(f'{i}-{j}')
        
data_agg.columns = data_agg_col

In [ ]:
corr_matrix = data_agg.corr()

upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

to_drop = [c for c in upper.columns if any(abs(upper[c]) > 0.9)]

print(f'There are {len(data_agg.columns)} columns but {len(to_drop)} corr columns to remove')

There are 300 columns but 150 corr columns to remove


In [ ]:
data_agg.drop(columns=to_drop)
data = data.merge(data_agg, how='left', on='id')

# 그냥 모델링 하기 다음에 피쳐 엔지니어링

In [ ]:
from sklearn.metrics import log_loss, make_scorer
import catboost
import lightgbm
import xgboost
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate, KFold, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, robust_scale

In [ ]:
x_train = data.loc[data['target'].notnull()].drop(columns=['id', 'target'])
y_train = data.loc[data['target'].notnull()]['target']
x_test = data.loc[data['target'].isnull()].drop(columns=['id', 'target'])

In [ ]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_train = pd.Series(y_train)

In [ ]:
x_train_col = x_train.columns
x_test_col = x_test.columns

x_train = pd.DataFrame(robust_scale(x_train), columns=x_train_col)
x_test = pd.DataFrame(robust_scale(x_test), columns=x_test_col)

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
preds = np.zeros((len(x_test)))
for i, (train_index, valid_index) in enumerate(skf.split(x_train, y_train)):
  X_train, X_valid = x_train.iloc[train_index], x_train.iloc[valid_index]
  Y_train, Y_valid = y_train.iloc[train_index], y_train.iloc[valid_index]

  model = lightgbm.LGBMClassifier(n_estimators=3000, learning_rate=0.05, objective='multiclass', n_jobs=-1)
  model.fit(X_train, Y_train, eval_set=[(X_train, Y_train), (X_valid, Y_valid)], verbose=100, early_stopping_rounds=70, eval_metric='multi_logloss')
  preds = model.predict_proba(x_test)

Training until validation scores don't improve for 70 rounds
[100]	training's multi_logloss: 1.66235	valid_1's multi_logloss: 1.75672
[200]	training's multi_logloss: 1.59642	valid_1's multi_logloss: 1.75755
Early stopping, best iteration is:
[137]	training's multi_logloss: 1.63543	valid_1's multi_logloss: 1.75649
Training until validation scores don't improve for 70 rounds
[100]	training's multi_logloss: 1.66408	valid_1's multi_logloss: 1.74879
[200]	training's multi_logloss: 1.59782	valid_1's multi_logloss: 1.74931
Early stopping, best iteration is:
[130]	training's multi_logloss: 1.64222	valid_1's multi_logloss: 1.74837
Training until validation scores don't improve for 70 rounds
[100]	training's multi_logloss: 1.6637	valid_1's multi_logloss: 1.75045
Early stopping, best iteration is:
[114]	training's multi_logloss: 1.65283	valid_1's multi_logloss: 1.75018
Training until validation scores don't improve for 70 rounds
[100]	training's multi_logloss: 1.66313	valid_1's multi_logloss: 1.7

In [ ]:
y_train.nunique()

9

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
preds_cat = np.zeros((x_test.shape[0], y_train.nunique()))
for i, (train_index, valid_index) in enumerate(skf.split(x_train, y_train)):
    X_train, X_valid = x_train.iloc[train_index], x_train.iloc[valid_index]
    Y_train, Y_valid = y_train.iloc[train_index], y_train.iloc[valid_index]
    
    model = catboost.CatBoostClassifier(n_estimators=3000, learning_rate=0.03, objective='MultiClass', eval_metric='MultiClass', task_type='GPU', devices='0:1')
    model.fit(X_train, Y_train, verbose=100, early_stopping_rounds=70 )
    preds_cat += model.predict_proba(x_test)/skf.n_splits


0:	learn: 2.1712908	total: 23ms	remaining: 1m 9s
100:	learn: 1.7524984	total: 2.02s	remaining: 58.1s
200:	learn: 1.7337598	total: 3.92s	remaining: 54.6s
300:	learn: 1.7245252	total: 5.66s	remaining: 50.8s
400:	learn: 1.7175564	total: 7.78s	remaining: 50.4s
500:	learn: 1.7115506	total: 9.89s	remaining: 49.3s
600:	learn: 1.7062127	total: 11.5s	remaining: 45.9s
700:	learn: 1.7012039	total: 13.1s	remaining: 42.9s
800:	learn: 1.6963311	total: 14.7s	remaining: 40.3s
900:	learn: 1.6916381	total: 16.3s	remaining: 37.9s
1000:	learn: 1.6873186	total: 18.1s	remaining: 36.1s
1100:	learn: 1.6829193	total: 19.6s	remaining: 33.9s
1200:	learn: 1.6786123	total: 21.2s	remaining: 31.8s
1300:	learn: 1.6744455	total: 22.8s	remaining: 29.8s
1400:	learn: 1.6704061	total: 24.4s	remaining: 27.8s
1500:	learn: 1.6662926	total: 26s	remaining: 25.9s
1600:	learn: 1.6621701	total: 27.6s	remaining: 24.1s
1700:	learn: 1.6582902	total: 29.4s	remaining: 22.4s
1800:	learn: 1.6541537	total: 31s	remaining: 20.6s
1900:	lear

In [ ]:
model_cat.select_features(
                X_train,
                Y_train,
                eval_set=[(X_train, Y_train), (X_valid, Y_valid)],
                features_for_select=None,
                num_features_to_select=None,
                algorithm='RecursiveByLossFunctionChange',
                steps=None,
                shap_calc_type=None,
                train_final_model=False,
                verbose=None,
                logging_level=None,
                plot=True)

In [ ]:
preds_cat

array([[0.05618264, 0.36730427, 0.15547635, ..., 0.02465586, 0.05812148,
        0.12116492],
       [0.04200647, 0.07692724, 0.05730919, ..., 0.08996002, 0.33041065,
        0.13009589],
       [0.02103344, 0.02642113, 0.02384407, ..., 0.0315424 , 0.12622487,
        0.05662751],
       ...,
       [0.07642187, 0.29309052, 0.12842135, ..., 0.0410427 , 0.11358795,
        0.13756208],
       [0.03616271, 0.01981402, 0.01977624, ..., 0.0669261 , 0.36978935,
        0.08328917],
       [0.03944433, 0.07657988, 0.07599119, ..., 0.08708266, 0.24721113,
        0.13636335]])

In [ ]:
from catboost.utils import get_gpu_device_count
get_gpu_device_count()

1

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
preds_xgb = np.zeros((len(x_test)))
for i, (train_index, valid_index) in enumerate(skf.split(x_train, y_train)):
  X_train, X_valid = x_train.iloc[train_index], x_train.iloc[valid_index]
  Y_train, Y_valid = y_train.iloc[train_index], y_train.iloc[valid_index]

  model_xgb = xgboost.XGBClassifier(n_estimators=3000, learning_rate=0.022, objective='multiclass')
  model_xgb.fit(X_train, Y_train, eval_set=[(X_train, Y_train), (X_valid, Y_valid)], verbose=100, early_stopping_rounds=70, eval_metric='mlogloss')
  preds_xgb = model_xgb.predict_proba(x_test)

[0]	validation_0-mlogloss:2.18486	validation_1-mlogloss:2.18537
[100]	validation_0-mlogloss:1.77213	validation_1-mlogloss:1.80677
[200]	validation_0-mlogloss:1.69942	validation_1-mlogloss:1.76371
[300]	validation_0-mlogloss:1.66592	validation_1-mlogloss:1.75669


KeyboardInterrupt: 

In [ ]:
submit = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/sample_submission.csv')
submit = submit.set_index('id')

In [ ]:
for i in range(9):
    submit.iloc[:,i] = preds_cat[:,i]

In [ ]:
submit.to_csv('submit8.csv')

# NN모델을 써야 점수가 올라간다. 
# Pytorch NN모델을 생성하고 제출하자
# feature_select할게 더 있나? 체크